<a href="https://colab.research.google.com/github/Vlastellin/HandTracking/blob/master/train_network_for_detect_3d_points.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


In [0]:
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model
import pickle
import os
import imageio
import numpy as np
from tensorflow.keras.preprocessing import image
import pickle

def preprocess_image(image_path):
  img = image.load_img(image_path)
  img = image.img_to_array(img)
  #print(img.shape)
  img = np.expand_dims(img, axis=0)
  #print(img.shape)
  #img = inception_v3.preprocess_input(img)
  return img

base_dir = '/content/drive/My Drive/find_3d_points'
imgs=[]
koords=[[],[],[]]
with open(os.path.join(base_dir, 'dataset_list.pickle'), 'rb') as f:
  l= pickle.load(f)
for i,koord in enumerate(l[1]):
  img_path=os.path.join(base_dir, 'dataset/'+str(i)+'.jpg')
  #image_raw = imageio.imread(img_path)
  #image_v = np.expand_dims((image_raw.astype('float') / 255.0) - 0.5, 0)
  imgs.append(preprocess_image(img_path)[0])
  #print(image_raw.shape)
  temp_koords=np.array([np.array([koord[0][0], koord[0][1],koord[0][2]]), np.array([koord[1][0], koord[1][1],koord[1][2]]),np.array([koord[2][0], koord[2][1],koord[2][2]])])
  for j,k in enumerate(temp_koords):
    koords[j].append(k)
  if i==799:
    break

imgs2=[]
koords2=[[],[],[]]
for i,img in enumerate(imgs):
  #print(i%8)
  if i%8==0:
    imgs2.append(img)
    koords2[0].append(koords[0][i])
    koords2[1].append(koords[1][i])
    koords2[2].append(koords[2][i])
imgs2 = np.array(imgs2)
koords2=np.array([koords2[0], koords2[1], koords2[2]])
with open(os.path.join(base_dir,'dataset2.pickle'), 'wb') as f:
     pickle.dump([imgs2, koords2], f)

In [0]:
for i in range(160):
  print(i%8)

In [20]:
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model
import pickle
import os
import imageio
import numpy as np
from tensorflow.keras.preprocessing import image


base_dir = '/content/drive/My Drive/find_3d_points'

input_img = Input(shape=(480,640, 3))
x = layers.Conv2D(32, (3, 3), activation='relu')(input_img)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Flatten()(x)
p1 = layers.Dense(3, name='p1')(x)
p2 = layers.Dense(3, name='p2')(x)
p3 = layers.Dense(3, name='p3')(x)
model = Model([input_img],[p1, p2, p3])
model.compile(optimizer='rmsprop',loss=['mae', 'mae', 'mae'])
#data

with open(os.path.join(base_dir,'dataset2.pickle'), 'rb') as f:
     l=pickle.load(f)
imgs=l[0]
koords=l[1]

    

print(imgs.shape)
print(imgs.ndim)
koords=np.array([koords[0], koords[1], koords[2]])
print(type(koords[0]))
#print(np.array(koords))
print(koords.shape)
from tensorflow.keras.utils import plot_model
#plot_model(model, to_file=os.path.join(base_dir,'model.png'), show_shapes=True)
#model.fit(x=imgs, y={'p0_x':koords[0], 'p0_y':koords[1], 'p0_z':koords[2],'p5_x':koords[3], 'p5_y':koords[4], 'p5_z':koords[5],'p9_x':koords[6], 'p9_y':koords[7], 'p9_z':koords[8]},epochs=2, batch_size=64)
model.fit(x=(imgs), y={'p1':koords[0], 'p2':koords[1], 'p3':koords[2]},epochs=9, validation_split=0.1, batch_size=150)
model.save(os.path.join(base_dir, "model1"))

(100, 480, 640, 3)
4
<class 'numpy.ndarray'>
(3, 100, 3)
Train on 90 samples, validate on 10 samples
Epoch 1/9
90/90 [==============================] - 34s 381ms/sample - loss: 253.5667 - p1_loss: 93.0741 - p2_loss: 81.4782 - p3_loss: 79.0145 - val_loss: 6633.3921 - val_p1_loss: 2232.0090 - val_p2_loss: 2132.1870 - val_p3_loss: 2269.1958
Epoch 2/9
90/90 [==============================] - 33s 364ms/sample - loss: 6276.2793 - p1_loss: 2112.9365 - p2_loss: 2030.6661 - p3_loss: 2132.6765 - val_loss: 235.0168 - val_p1_loss: 98.0688 - val_p2_loss: 69.8174 - val_p3_loss: 67.1306
Epoch 3/9
90/90 [==============================] - 33s 368ms/sample - loss: 223.3909 - p1_loss: 92.8927 - p2_loss: 68.0182 - p3_loss: 62.4799 - val_loss: 969.0040 - val_p1_loss: 313.7039 - val_p2_loss: 327.7005 - val_p3_loss: 327.5997
Epoch 4/9
90/90 [==============================] - 33s 364ms/sample - loss: 916.0747 - p1_loss: 297.7280 - p2_loss: 306.2825 - p3_loss: 312.0642 - val_loss: 268.9476 - val_p1_loss: 107.4

In [10]:
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model
import pickle
import os
import imageio
import numpy as np
from tensorflow.keras.preprocessing import image


def preprocess_image(image_path):
  img = image.load_img(image_path)
  img = image.img_to_array(img)
  #print(img.shape)
  img = np.expand_dims(img, axis=0)
  #print(img.shape)
  #img = inception_v3.preprocess_input(img)
  return img

base_dir = '/content/drive/My Drive/find_3d_points'
imgs=[]
koords=[[],[],[]]
with open(os.path.join(base_dir, 'dataset_list.pickle'), 'rb') as f:
  l= pickle.load(f)
for i,koord in enumerate(l[1]):
  img_path=os.path.join(base_dir, 'dataset/'+str(i)+'.jpg')
  #image_raw = imageio.imread(img_path)
  #image_v = np.expand_dims((image_raw.astype('float') / 255.0) - 0.5, 0)
  imgs.append(preprocess_image(img_path)[0])
  #print(image_raw.shape)
  temp_koords=np.array([np.array([koord[0][0], koord[0][1],koord[0][2]]), np.array([koord[1][0], koord[1][1],koord[1][2]]),np.array([koord[2][0], koord[2][1],koord[2][2]])])
  for j,k in enumerate(temp_koords):
    koords[j].append(k)
  if i==150:
    break

for i,img in enumerate(imgs):
  prediction=model.predict(np.array([img]))
  print(koords[0][i], koords[1][i], koords[2][i])
  print(prediction)

model = models.load_model(MODEL_FILENAME)

[-52.9282 148.187  -77.0002] [-23.0833 139.466  -40.6422] [ 17.3511 131.42   -32.2729]
[array([[-59.029408, 164.34898 , -64.869675]], dtype=float32), array([[-37.844353, 168.2546  , -26.922783]], dtype=float32), array([[ 19.271925, 170.73912 , -36.663826]], dtype=float32)]
[-52.9282 148.187  -77.0002] [-23.0833 139.466  -40.6422] [ 17.3511 131.42   -32.2729]
[array([[-63.255013, 182.45448 , -71.166435]], dtype=float32), array([[-39.473423, 188.71205 , -32.87105 ]], dtype=float32), array([[ 20.176186, 189.87373 , -39.640297]], dtype=float32)]
[-52.9282 148.187  -77.0002] [-23.0833 139.466  -40.6422] [ 17.3511 131.42   -32.2729]
[array([[-64.15214 , 174.57571 , -66.728935]], dtype=float32), array([[-41.237675, 178.76257 , -31.505007]], dtype=float32), array([[ 19.118465, 182.56117 , -38.666874]], dtype=float32)]
[-52.9282 148.187  -77.0002] [-23.0833 139.466  -40.6422] [ 17.3511 131.42   -32.2729]
[array([[-64.87209, 173.6385 , -67.03789]], dtype=float32), array([[-40.714287, 178.10397 ,

NameError: ignored

In [0]:
!pip install -U numpy


Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (1.17.4)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
